In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Fixed parameters (number of samples)

In [2]:
#Number of samples determined by Chebyshev inequality
samples = 10000

# Variable parameters (qubits and layers)

In [3]:
#List of number of qubits
qubits = [3]

#List of number of layers
layers = [5]

# Function to calculate Meyer-Wallach mean and standard deviation

In [4]:
def ent_cap(samples, circuit, nlayers):
    MW_mean = 0
    MW_mean_sqr = 0
    for i in range(samples):
        angles = np.array([ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)], 
                            [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] 
                            for j in range(nlayers)], requires_grad=True)
        
        pur_sum = 0
        for k in range(wires):
            state = circuit(angles, k)
            pur_sum = pur_sum + np.trace(np.matmul(state, state))
    
        MW_measure = 2*(1-1/wires*pur_sum)
        MW_measure_sqr = MW_measure**2
        
        MW_mean = MW_mean + MW_measure
        MW_mean_sqr = MW_mean_sqr + MW_measure_sqr
        
    MW_mean = MW_mean/samples
    MW_mean_sqr = MW_mean_sqr/samples
    MW_standev = np.sqrt(MW_mean_sqr - MW_mean**2)
    return(MW_mean, MW_standev)

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")


for wires in qubits:
    dev = qml.device("default.qubit", wires=wires, shots=None)
    
    for nlayers in layers:
        
        #Identity circuit
        def identity(rots, k):
            for i in range(wires):
                qml.Identity(wires = [i])
            return qml.density_matrix([k])

        identity_circuit = qml.QNode(identity, dev)


        
        #No connections circuit
        def layer_noconnec(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
        
        
        def noconnec(rotations, k):
            qml.layer(layer_noconnec, nlayers, rotations)
            return qml.density_matrix([k])

        noconnec_circuit = qml.QNode(noconnec, dev)
        
        
        
        #Linear circuit
        def layer_linear(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")   

        def linear(rotations, k):
            qml.layer(layer_linear, nlayers, rotations)
            return qml.density_matrix([k])

        linear_circuit = qml.QNode(linear, dev)
        
        
        
        #Ring circuit
        def layer_ring(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
            qml.CNOT(wires=[wires-1,0]) 

        def ring(rotations, k):
            qml.layer(layer_ring, nlayers, rotations)
            return qml.density_matrix([k])

        ring_circuit = qml.QNode(ring, dev)        
        
        
        
        ent_identity, ent_identity_standev  = ent_cap(samples, identity_circuit, nlayers)
        ent_noconnec, ent_noconnec_standev = ent_cap(samples, noconnec_circuit, nlayers)
        ent_linear, ent_linear_standev = ent_cap(samples, linear_circuit, nlayers)
        ent_ring, ent_ring_standev = ent_cap(samples, ring_circuit, nlayers)
        
        f = open("entanglement_ansatz1_qubits{0}_layers{1}_samples{2}.txt".format(wires,nlayers, samples), "w")

        print('Circuit,MW_mean,MW_standdev', file = f)
        print('Identity,', ent_identity,',', ent_identity_standev, file = f)
        print('No connections,',ent_noconnec,',', ent_noconnec_standev, file = f)
        print('Linear,',ent_linear,',', ent_linear_standev, file = f)
        print('Ring,',ent_ring,',', ent_ring_standev, file = f)
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        print()
        

now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Execution: qubits= 3 ; layers= 5

Tempo inicial: 
2023-07-10 16:05:43
Tempo final:
2023-07-10 16:40:51


In [6]:
f.close()